In [2]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import cv2
import os

In [4]:
def create_graph():
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile(os.path.join(
      '/data/models/inception/', 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

In [6]:
embedding_list = []
labels_list = []
#sess = tf.Session('local', graph=tf.get_default_graph())
#with sess, sess.graph.device(tf.ReplicaDeviceSetter(ps_tasks=0, worker_device='/gpu:0')):
# Creates graph from saved GraphDef.
create_graph()
c = 0
with tf.Session() as sess:
    # Some useful tensors:
    # 'softmax:0': A tensor containing the normalized prediction across
    #   1000 labels.
    # 'pool_3:0': A tensor containing the next-to-last layer containing 2048
    #   float description of the image.
    # 'DecodeJpeg/contents:0': A tensor containing a string providing JPEG
    #   encoding of the image.
    # Runs the softmax tensor by feeding the image_data as input to the graph.
    embedding_tensor = sess.graph.get_tensor_by_name('pool_3:0')

    for line in open('/data/gt_proposals/info.txt'):
        fields = line.split(',')
        if int(fields[3]) == 1:
            image_data = tf.gfile.FastGFile(fields[0], 'rb').read()
            embeddings = sess.run(embedding_tensor, {'DecodeJpeg/contents:0': image_data})
            labels_list.append(int(fields[2]))
            embedding_list.append(embeddings)
            c += 1

In [11]:
labels_list = np.array(labels_list).reshape(c, 1)
embedding_list = np.array(embedding_list)
embedding_list = embedding_list.reshape(c, 2048)

In [12]:
print labels_list.shape
print embedding_list.shape

(5257, 1)
(5257, 2048)


In [13]:
save_array = np.hstack((labels_list, embedding_list))
print save_array.shape
np.save('/data/gt_proposals/embeddings_baseline.npy', save_array)

(5257, 2049)
